In [1]:
import pandas as pd 
import numpy as np
from datasets import load_dataset
from NetfoundTokenizer import NetFoundTokenizer
from NetfoundConfig import NetfoundConfig


In [14]:
config = NetfoundConfig(
    num_hidden_layers=12,
    no_meta=False,
    flat=False,
)

In [15]:
tokenizer = NetFoundTokenizer(config=config)

In [6]:
train_dir="/global/homes/h/haarika/pscratch/test_directories/speedtest_debug/tokens"

In [4]:
train_dir="/global/homes/h/haarika/pscratch/raw_speedtest_pcaps_ipv4/tokens_new_run"

In [7]:
train_dataset = load_dataset(
    "arrow",
    data_dir=train_dir
)

 

Resolving data files:   0%|          | 0/12229 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/25 [00:00<?, ?it/s]

In [8]:
train_dataset = train_dataset.remove_columns(['labels'])


In [9]:
train_dataset

DatasetDict({
    train: Dataset({
        features: ['flow_duration', 'burst_tokens', 'directions', 'bytes', 'iats', 'rts', 'protocol'],
        num_rows: 12400
    })
})

In [ ]:
train_dataset_new = train_dataset.map(tokenizer, batched=True,load_from_cache_file=False, num_proc=32)


Map (num_proc=32):   0%|          | 0/12400 [00:00<?, ? examples/s]

TimeoutError: 

: 

In [ ]:
train_dataset[]

DatasetDict({
    train: Dataset({
        features: ['flow_duration', 'burst_tokens', 'directions', 'bytes', 'iats', 'rts', 'protocol', 'labels'],
        num_rows: 12400
    })
})

In [11]:
train_dataset_new=train_dataset.select(range(6))

AttributeError: 'DatasetDict' object has no attribute 'select'

In [ ]:
    tokenizer = NetFoundTokenizer(config=config)


In [3]:
def load_train_test_datasets(logger, data_args):
    logger.warning("Loading datasets")
    train_split = test_split = None
    if data_args.test_dir is None:
        data_args.test_dir = data_args.train_dir
        train_split = f"train[{data_args.validation_split_percentage}%:]"
        test_split = f"train[:{data_args.validation_split_percentage}%]"

    train_dataset = load_dataset(
        "arrow",
        data_dir=data_args.train_dir,
        split=train_split,
        cache_dir=data_args.data_cache_dir,
    )

    test_dataset = load_dataset(
        "arrow",
        data_dir=data_args.test_dir,
        split=test_split,
        cache_dir=data_args.data_cache_dir,
    )

    if data_args.max_eval_samples is not None:
        test_dataset = test_dataset.select(
            range(min(test_dataset.shape[0], data_args.max_eval_samples))
        )
    if data_args.max_train_samples is not None:
        train_dataset = train_dataset.select(
            range(min(train_dataset.shape[0], data_args.max_train_samples))
        )
    train_dataset = train_dataset.add_column("total_bursts", [0] * len(train_dataset))
    test_dataset = test_dataset.add_column("total_bursts", [0] * len(test_dataset))

    return train_dataset, test_dataset